<a href="https://colab.research.google.com/github/theDreamer911/dailyChallenges/blob/master/Gender_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in case you want to download the dataset

[links](https://drive.google.com/file/d/1AwhJ41jsQwYosP8Ak7tO7ldJILTkuGz8/view?usp=sharing)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/dataset"

gender_classifier.zip


In [6]:
!unzip "/content/drive/My Drive/dataset/gender_classifier.zip"

Archive:  /content/drive/My Drive/dataset/gender_classifier.zip
replace testing/face/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace testing/face/2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: YNA
  inflating: testing/face/2.jpg      
replace testing/face/3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: testing/face/3.jpg      
  inflating: testing/face/4.jpg      
  inflating: testing/face/5.jpg      
  inflating: testing/face/6.jpg      
  inflating: testing/face/7.jpg      
  inflating: testing/face/8.jpg      
  inflating: train/1female/0.jpg     
  inflating: train/1female/1.jpg     
  inflating: train/1female/10.jpg    
  inflating: train/1female/100.jpg   
  inflating: train/1female/101.jpg   
  inflating: train/1female/102.jpg   
  inflating: train/1female/103.jpg   
  inflating: train/1female/104.jpg   
  inflating: train/1female/105.jpg   
  inflating: train/1female/106.jpg   
  inflating: train/1female/107.jpg   
  inflating: train/1female/108.jpg   
  inflat

## Dependencies and Global Variable

In [8]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [9]:
# Image Dimensions
img_width, img_height = 128, 128

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

In [10]:
# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

## Data Augmentation

In [12]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [13]:
# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 800 images belonging to 2 classes.


In [15]:
# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 240 images belonging to 2 classes.


## KiiTa Model
*Intinya model bikinan sendiri*

In [16]:
# Feature Extraction Layer
inputs = Input(shape=(img_width, img_height, 3))
conv_layer = Conv2D(16, (5,5), strides=(3,3), activation='relu')(inputs)
conv_layer = ZeroPadding2D(padding=(1,1))(conv_layer)
conv_layer = Conv2D(32, (5,5), strides=(3,3), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2,2))(conv_layer)
conv_layer = Conv2D(64, (3,3), strides=(1,1), activation='relu')(conv_layer)
conv_layer = Conv2D(64, (3,3), strides=(1,1), activation='relu')(conv_layer)

In [17]:
# Flatten Layer
flatten = Flatten()(conv_layer)

In [18]:
# Fully Connected Layer
fc_layer = Dense(32, activation='relu')(flatten)
outputs = Dense(1, activation='sigmoid')(fc_layer)

model = Model(inputs = inputs, outputs = outputs)

In [19]:
# Adam Optimizer and Cross Entropy Loss
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Print Model Summary
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 42, 42, 16)        1216      
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 44, 44, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928 

In [21]:
model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = [callbacks]
)

model.save_weights('kiita.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
50/50 [==============================] - 7s 122ms/step - loss: 0.6927 - accuracy: 0.5088 - val_loss: 0.6898 - val_accuracy: 0.5958
Epoch 2/50
50/50 [==============================] - 6s 118ms/step - loss: 0.6845 - accuracy: 0.6539 - val_loss: 0.6823 - val_accuracy: 0.4875
Epoch 3/50
50/50 [==============================] - 6s 118ms/step - loss: 0.6738 - accuracy: 0.5894 - val_loss: 0.6623 - val_accuracy: 0.6500
Epoch 4/50
50/50 [==============================] - 6s 117ms/step - loss: 0.6493 - accuracy: 0.6784 - val_loss: 0.6478 - val_accuracy: 0.5833
Epoch 5/50
50/50 [==============================] - 6s 118ms/step - loss: 0.6262 - accuracy: 0.6589 - val_loss: 0.6252 - val_accuracy: 0.6583
Epoch 6/50
50/50 [==============================] - 6s 118ms/step - loss: 0.6198 - accuracy: 0.6597 - val_loss: 0.6229 - val_accuracy: 0.6833
Epoch 7/50
50/50 [==============================] - 6s 117ms/step - loss: 0.5971 - accuracy: 0.7023 - val_loss: 0.6054 - val_accuracy: 0.6833
Epoch 

## VGG-16 Dependecies and Variable

In [22]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [23]:
# Image Dimensions
img_width, img_height = 128, 128

In [24]:
# Data
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 800
nb_validation_samples = 240
epochs = 50
batch_size = 16

In [25]:
# TensorBoard Callbacks
callbacks = TensorBoard(log_dir='./Graph')

## VGG-16 Model dan Augmentasi Data

In [26]:
# Build VGG16
model = applications.VGG16(include_top=False, weights='imagenet')

58892288/58889256 [==============================] - 1s 0us/step


In [27]:
# Training Data Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [28]:
# Rescale Testing Data
test_datagen = ImageDataGenerator(rescale = 1. / 255)

In [29]:
# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

train_features = model.predict_generator(
    train_generator, nb_train_samples // batch_size, verbose = 1
)

np.save('train_features.npy', train_features)

Found 800 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


50/50 [==============================] - 117s 2s/step


In [32]:
# Testing Data Generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_features = model.predict_generator(
    validation_generator, nb_validation_samples // batch_size, verbose=1
)

np.save('val_features.npy', validation_features)

Found 240 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


15/15 [==============================] - 34s 2s/step


## VGG-16 FC Layer

In [33]:
# Build Train Data
train_data = np.load('train_features.npy')
train_labels = np.array(
    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples //2 )
)

In [34]:
# Build Validation Data
validation_data = np.load('val_features.npy')
validation_labels = np.array(
    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2)
)

In [35]:
# Build FC Layer
fc_model = Sequential()
fc_model.add(Flatten(input_shape = train_data.shape[1:]))
fc_model.add(Dense(32, activation='relu'))
fc_model.add(Dense(1, activation='sigmoid'))

In [36]:
# Adam Optimizer and Cross Entropy Loss
adam = Adam(learning_rate=0.0001)
fc_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
fc_model.fit(train_data, train_labels,
             epochs = epochs,
             batch_size = batch_size,
             validation_data = (validation_data, validation_labels),
             callbacks=[callbacks])

fc_model.save_weights('vgg16.h5')

Epoch 1/50
50/50 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9975 - val_loss: 1.5710 - val_accuracy: 0.5292
Epoch 2/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 1.5955 - val_accuracy: 0.5375
Epoch 3/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0071 - accuracy: 0.9987 - val_loss: 1.5922 - val_accuracy: 0.5125
Epoch 4/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 1.6177 - val_accuracy: 0.5500
Epoch 5/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0068 - accuracy: 0.9987 - val_loss: 1.6058 - val_accuracy: 0.5417
Epoch 6/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0074 - accuracy: 0.9975 - val_loss: 1.6235 - val_accuracy: 0.5083
Epoch 7/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0071 - accuracy: 0.9987 - val_loss: 1.6294 - val_accuracy: 0.5458
Epoch 8/50
50/50 [==

## Bonus Experiment

In [40]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import Adam

In [41]:
img_width, img_height = 128, 128

test_data_dir = 'testing'

In [44]:
def get_features():
  datagen = ImageDataGenerator(rescale=1. / 255)

  # Build VGG16 Network
  model = applications.VGG16(include_top=False, weights='imagenet')

  generator = datagen.flow_from_directory(
      test_data_dir,
      target_size=(img_width, img_height),
      batch_size=1,
      class_mode='binary',
      shuffle=False
  )

  features = model.predict_generator(
      generator, 8
  )

  return features

def get_score():
  features = get_features()

  model = Sequential()
  model.add(Flatten(input_shape = features.shape[1:]))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.load_weights('vgg16.h5')

  score = model.predict(features)
  print(score)

get_score()

Found 8 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[[0.0016399 ]
 [0.99488676]
 [0.60782975]
 [0.00175485]
 [0.07274881]
 [0.8296595 ]
 [0.25419092]
 [0.07329875]]
